# River tracers

A visualisation of passive river tracers for the Don, O'Connell, and Pioneer rivers near the Whitsundays.
Passive river tracer results derived from version 2.0 of the 1km-resolution shelf-scale hydrodynamic model of the Great Barrier Reef (GBR1).

In [ ]:
import cartopy
import emsarray
import matplotlib.colors
import matplotlib.figure
import matplotlib.pyplot as plt
import matplotlib.quiver
import numpy
import pandas
import shapely
import xarray

# width, height in pixels
IMAGE_SIZE = (1000, 1000)
DPI = 100 # matplotlib default

# The plot will cover this region, expanding as required to match the aspect ratio set in IMAGE_SIZE.
# The coordinates represent the (west, east, south, north) sides of a region that will be plotted.
MAP_EXTENT = (147.80, 149.60, -21.35, -19.55)

RIVERS_URL = 'https://thredds.nci.org.au/thredds/dodsC/fx3/model_data/gbr1_2.0_rivers.ncml'
RIVERS_TIMESTEP = '2022-03-01T14:00:00.00000000'

MODIS_TIMESTEP = '2022-03-01'

Open the GBR1 v2.0 rivers dataset and select one time step at the ocean surface.

In [ ]:
rivers_dataset = emsarray.open_dataset(RIVERS_URL, decode_timedelta=False)

rivers_dataset = rivers_dataset.set_coords(['time', 'zc'])
rivers_dataset = rivers_dataset.set_xindex(['zc'])
rivers_dataset = rivers_dataset.sel(time=RIVERS_TIMESTEP, zc=0.5)

Select the rivers we are interested in and scale the values to make the finer detail more prominent.

In [ ]:
# Collect the Don, O'Connell, and Pioneer river tracers
river_names = [
    'don', # Don River
    'con', # O'Connell River
    'pio', # Pioneer River
]
rivers_dataset = rivers_dataset.ems.select_variables(river_names)
rivers_dataset.load()
convention = rivers_dataset.ems

# Make a DataArray with each of the rivers stacked along a new dimension 'tracer'.
river_tracers = xarray.concat([
    rivers_dataset[name].assign_coords({'tracer': name})
    for name in river_names
], dim='tracer')
river_tracers.name = 'tracers'

# Find the maximum tracer concentration across the entire domain.
# Scale this maximum down - the tracer is most concentrated at the river
# mouth and dissipates quickly, so most of the interesting detail is in the
# lower numbers.
river_max = river_tracers.max() / 32

# Normalise the tracer concentration using a power function.
# This will enhance lower concentrations.
# clip=True is required to keep values between 0 and 1 as we scaled the river max down
norm = matplotlib.colors.PowerNorm(0.3, vmin=0, vmax=river_max, clip=True)
river_tracers = xarray.DataArray(
    data=norm(river_tracers.values),
    dims=river_tracers.dims,
    coords=river_tracers.coords
)

# Scale it such that the maximum tracer sum in any cell is 1
# river_tracers = river_tracers / river_tracers.sum(dim='tracer').max()

Assign a colour to each river, plus a colour for the ocean

In [ ]:
# Assume that anything that isn't tracer is ocean water
ocean = 1 - river_tracers.sum(dim='tracer')
ocean = xarray.where(numpy.isnan(river_tracers).any(dim='tracer'), numpy.nan, ocean)
ocean = ocean.assign_coords({'tracer': 'ocean'})
all_tracers = xarray.concat([river_tracers, ocean], dim='tracer')

# The colours for the river tracers and for ocean water.
# I've tried a few different colour schemes. This is the best I've come up with.
# Feel free to experiment!
colour_dark = [
    [127, 191, 0, 255],
    [191, 0, 127, 255],
    [0, 127, 191, 255],
    [0, 0, 0, 255],
]
colour_light = [
    [64, 128, 0, 255],
    [128, 0, 64, 255],
    [0, 64, 128, 255],
    [255, 255, 255, 255],
]
ocean_colours = xarray.DataArray(
    data=colour_dark,
    dims=['tracer', 'component'],
    coords={
        'tracer': all_tracers.coords['tracer'],
        'component': ['r', 'g', 'b', 'a'],
    },
)
ocean_colours = ocean_colours / 255

Plot the river tracers overlayed on MODIS true colour satellite imagery from NASA GIBS.

In [ ]:
# Set up the figure for plotting
figure = plt.figure(
    figsize=tuple(i / DPI for i in IMAGE_SIZE),
    dpi=DPI,
    layout='constrained')

axes = figure.add_subplot(projection=cartopy.crs.PlateCarree())
plt.title("River tracer concentration")
    
# Cover the map extent with the plot, expanding the extent as necessary to
# keep the image aspect ratio.
axes.set_extent(MAP_EXTENT)
axes.set_aspect('equal', adjustable='datalim')

# Add MODIS baselayer from NASA GIBS
axes.add_wms(
    wms='https://gibs.earthdata.nasa.gov/wms/epsg4326/best/wms.cgi',
    layers=['MODIS_Terra_CorrectedReflectance_TrueColor'],
    wms_kwargs={'time': MODIS_TIMESTEP},
)

cell_colours = (all_tracers * ocean_colours).sum(dim='tracer')
cell_colours = cell_colours.clip(min=0., max=1.)
cell_colours = convention.ravel(cell_colours).transpose()[convention.mask].values
axes.add_collection(convention.make_poly_collection(
    color=cell_colours,
    edgecolor="face",
))

river_patches = [
    matplotlib.patches.Patch(
        color=river.values,
        label=rivers_dataset[river.coords['tracer'].values.item()].attrs['long_name'],
    )
    for river in ocean_colours[ocean_colours.coords['tracer'] != 'ocean'].transpose('tracer', ...)
]
axes.legend(handles=river_patches, loc='lower left')

axes.text(
    0.99, 0.01, "Satellite imagery provided by NASA's Global Imagery Browse Services (GIBS),\npart of NASA's Earth Science Data and Information System (ESDIS).",
    transform=axes.transAxes,
    horizontalalignment='right', verticalalignment='bottom',
    bbox=dict(facecolor='white', alpha=0.5, edgecolor='black', linewidth=1),
)
